In [1]:
import pandas as pd   #used for data manipulation and analysis
import numpy as np    #used for mathematical and logical operations

In [2]:
data = pd.read_csv('dataset_bank.csv')
data.head()

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39.0,...,1,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44.0,...,1,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105
2,713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36.0,...,1,0,3418.0,0,3418.0,2.594,1887,20,2.333,0.000
3,769911858,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34.0,...,4,1,3313.0,2517,796.0,1.405,1171,20,2.333,0.760
4,709106358,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21.0,...,1,0,4716.0,0,4716.0,2.175,816,28,2.500,0.000


In [3]:
data.iloc[10].to_csv("single_line.csv")

In [4]:
data[1:2].to_csv('sample.csv', index=False)

In [5]:
sample = pd.read_csv('sample.csv')
df=sample

### Gore cita edna linija

## 1 
#std, scale, remove missing, i.t.n => same as best model
#preprocessed_sample = ......

In [6]:
#Delime na 2 mnozestva. prvoto>
df_num = df.select_dtypes(include = ['float64', 'int64'])

In [7]:
# tuka se kategoriski , vtoroto:
df_cat = df.select_dtypes(include = ['object'])

In [8]:
df_num['Months_on_book']=df_num.Months_on_book.replace({-2147483647.0:np.nan,
                                                       2147483647.0:np.nan,
                                                       3.21000000e+11:np.nan})

<ipython-input-8-5ca36817dfc6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_num['Months_on_book']=df_num.Months_on_book.replace({-2147483647.0:np.nan,


In [9]:
import sklearn
from sklearn.impute import SimpleImputer

In [10]:
mob_imputer = SimpleImputer(missing_values = np.NaN, strategy = 'mean') # go definirame Simple imputer, stratgegijata = mean
df_num.Months_on_book = mob_imputer.fit_transform(df_num['Months_on_book'].values.reshape(-1,1))[:,0]

C:\Users\Goran\anaconda3\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [11]:
from sklearn import preprocessing
from sklearn.preprocessing import RobustScaler # za varijablite so outleeri

In [12]:
df_num_rs = df_num.copy()

In [13]:

robscal = RobustScaler()
columnsO = ['Months_on_book','Credit_Limit', 'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt', 'Total_Ct_Chng_Q4_Q1']

df_num_rs[columnsO] = robscal.fit_transform(df_num_rs[columnsO])

In [14]:
from sklearn.preprocessing import StandardScaler # za varijablite bez outleeri
standscal = StandardScaler()

In [15]:
columnsNO = ['Customer_Age', 'Dependent_count', 'Total_Relationship_Count', 'Months_Inactive_12_mon', 'Contacts_Count_12_mon',
            'Total_Revolving_Bal', 'Total_Trans_Ct', 'Avg_Utilization_Ratio']

df_num_rs[columnsNO] = standscal.fit_transform(df_num_rs[columnsNO])

In [16]:
target = df.Attrition_Flag
target = target.replace({'Existing Customer': '0', 'Attrited Customer': '1'})
y1 = target.copy()
df_cat_cb = df_cat.copy()

In [17]:
#skaliranje so CatBoost
import category_encoders as ce

cat_columns = ['Gender', 'Education_Level', 'Marital_Status', 'Income_Category','Card_Category']

catBoostEncoder = ce.CatBoostEncoder(cols=cat_columns)

df_cat_cb = catBoostEncoder.fit_transform(df_cat_cb, y1)

C:\Users\Goran\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


Here we merge ....

In [18]:
# Gi spojuva
frames = [df_num_rs, df_cat_cb]

X_cb = pd.concat(frames, axis = 1)

In [33]:
preprocessed_sample = X_cb.drop(['CLIENTNUM', 'Attrition_Flag', 'Avg_Open_To_Buy','Credit_Limit','Customer_Age','Dependent_count','Months_on_book'], axis = 1)
# X1_new_train

In [34]:
preprocessed_sample.shape

(1, 14)

In [20]:
!pip install xgboost
import xgboost as xgb
from xgboost import XGBClassifier

## 2
#best_model = picke.load(.)

In [26]:
# import pickle
# filename='XGB_best_model'

# pickle.dump(best_model, open(filename, 'wb'))


NameError: name 'best_model' is not defined

In [35]:
filename = 'XGB_best_model'
XGB_best_model = pickle.load(open(filename, 'rb'))

# 3
XGB_best_model.predict(preprocessed_sample)

In [36]:
preprocessed_sample.shape

(1, 14)

In [37]:
XGB_best_model.predict(preprocessed_sample)

array(['0'], dtype=object)